# Problem of Expected number of games

If we have coin game with odds of 70% and amount of win 60% of bet, what part of many must be bet to get lowest losing probability?

Before solving actual mathematical problem, let's try numberical variations. In fact mathematical problem is so hard and final formula so complicated that it may be useless.

In [2]:
# imports and defentitions:
import numpy as np
import tqdm
import sympy
import pandas as pd
import statistics

def list_creator (p, w, n, Bstart, t ):
    stop_locations = []
    for c in tqdm.tqdm(range(t)):
        B = Bstart
        k = 1
        while True:
            if B < 1:
                stop_locations.append(k)
                break
            bet = B/n
            if bet < 1:
                bet = 1

            if np.random.choice((True, False), p=[p, 1-p]):
                B += w*bet
            else:
                B -= bet
            k += 1
    return stop_locations

def win_list_creator (p, w, n, Bstart, t ):
    stop_locations = []
    for c in tqdm.tqdm(range(t)):
        B = Bstart
        k = 1
        while True:
            if B < 1:
                break
            if B >= 2*Bstart:
                stop_locations.append(k)
                break
            bet = B/n
            if bet < 1:
                bet = 1

            if np.random.choice((True, False), p=[p, 1-p]):
                B += w*bet
            else:
                B -= bet
            k += 1
    return stop_locations

def win_lose (p, w, n, Bstart, t ):
    win = 0
    lose = 0
    for c in range(t):
        B = Bstart
        k = 1
        while True:
            if B < 1:
                lose += 1
                break
            if B >= 2*Bstart:
                win += 1
                break
            bet = B/n
            if bet < 1:
                bet = 1

            if np.random.choice((True, False), p=[p, 1-p]):
                B += w*bet
            else:
                B -= bet
            k += 1
    return (win, lose)


Suppose we start with $n=2$ and initial capital is $10,000\$$
Than we can generate list of number of games after which initial capital drops below 1. If size of list is big enough we can calculate average number of games after which initial capital will drop below 1.

In [3]:
l2 = list_creator(0.7, 0.6, 2, 10000, 100000)
%store l2

100%|██████████| 100000/100000 [18:24<00:00, 90.58it/s]

Stored 'l2' (list)


after we generated list we can apply statistics to it.

In [74]:
%store -r l2
print('mean', statistics.mean(l2))
print('standard deviation', statistics.stdev(l2))
print('variance', statistics.variance(l2))

mean 399.67917
standard deviation 364.51615755464434
variance 132872.02911840228


Eventuly if we don't decide were to stop, or don't define win condition, probability that initial capital will go below 1 is $100\%$ . So, defining that winning is doubling initial capital, we can calculate average number of games needed for this, by same method as previus calculations.

In [13]:
wl2 = win_list_creator(0.7,0.6, 2, 10000, 100000)
%store wl2

100%|██████████| 100000/100000 [02:17<00:00, 727.31it/s]

Stored 'wl2' (list)


In [14]:
%store -r wl2
print('mean', statistics.mean(wl2))
print('standard deviation', statistics.stdev(wl2))
print('variance', statistics.variance(wl2))

mean 25.654011290948464
standard deviation 52.42092157672384
variance 2747.953018953031


As we see mean with win is lower and it is better, but it does't tell us much about probability to win. So another function will calculte times win and compare with number of loses.

In [17]:
wln2 = win_lose(0.7, 0.6, 2, 10000, 100000)
%store wln2 

100%|██████████| 100000/100000 [02:23<00:00, 696.32it/s]


Stored 'wln2' (tuple)


Percentage will be win divided by total number.

In [18]:
%store -r wln2
wln2[0] / sum(wln2)

0.79952

Initilay we where interested in finding best n, which will minimaze the posibility of losing, or in other words maximaze winning ratio. So, we can take this formula, substitude different values for n and see what happends. Also, we can change inital capital and observe diferences.

In [5]:
index = [2,3,4,5,6,7,8,9,10,11]
columns = [10000, 5000, 2500, 1250, 625, 300, 150, 100, 50, 20]
pd_table1 = pd.DataFrame(None, columns=columns, index=index)

with tqdm.tqdm(total = 100) as bar:
    for column in columns:
        for idx in index:
            cuv = win_lose(0.7, 0.6, idx, column, 100000)
            pd_table1[column][idx] = round(100 * cuv[0]/ sum(cuv), 2)
            bar.write(str(round(100 * cuv[0]/ sum(cuv), 2)))
            bar.update(1)

%store pd_table1

  1%|          | 1/100 [02:17<3:47:18, 137.76s/it]

79.45


  2%|▏         | 2/100 [05:21<4:29:16, 164.86s/it]

96.75


  3%|▎         | 3/100 [08:10<4:29:33, 166.73s/it]

...

99.58


 99%|█████████▉| 99/100 [3:48:18<02:23, 143.87s/it]

99.71


100%|██████████| 100/100 [3:51:16<00:00, 138.77s/it]

99.82
Stored 'pd_table1' (DataFrame)


In [7]:
%store -r pd_table1
pd_table1

,10000,5000,2500,1250,625,300,150,100,50,20
2,79.45,79.37,79.19,78.78,77.86,77.44,76.44,75.7,74.5,72.09
3,96.75,96.35,95.68,95.07,94.33,93.05,91.69,90.89,88.77,85.0
4,99.79,99.74,99.63,99.39,99.07,98.56,97.84,97.21,95.64,92.34
5,99.99,99.99,99.97,99.95,99.89,99.75,99.5,99.25,98.51,96.12
6,100.0,100.0,100.0,100.0,99.99,99.97,99.86,99.8,99.42,97.94
7,100.0,100.0,100.0,100.0,100.0,99.99,99.98,99.94,99.79,98.86
8,100.0,100.0,100.0,100.0,100.0,100.0,99.99,99.98,99.92,99.3
9,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.97,99.58
10,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.99,99.71
11,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,99.99,99.82
